In [1]:
import numpy as np
import random
import json
import os
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
from collections import Counter
%load_ext autoreload
%autoreload 2

In [4]:
init_sources ={
    'random': 'random_init/',
    'half_half': 'half_half_init/',
    'full_heuristic': 'full_heuristic_init/'
}

model_sources = {
    'random': '../heuristic_results2/pickle_random_model/',
    'half_half': '../heuristic_results3/pickle_model/',
    'full_heuristic': '../heuristic_results4/pickle_full_model/'
}

depths = [3, 5, 10, 13]
GA_strats = [[50, 30, 20], [40, 40, 20], [30, 40, 30], [20, 50, 30], [10, 50, 40]]

In [5]:
best_5_models = {}
best_5_model_objects = {}

split_features = {}
#split_values = {}
max_depth_dict = {}

which_generation = 10

for init_type, init_source in init_sources.items():
    best_5_models[init_type] = {}
    best_5_model_objects[init_type] = {}
    split_features[init_type] = {}
    max_depth_dict[init_type] = {}
    
    for d in depths:
        best_5_models[init_type][d] = {}
        best_5_model_objects[init_type][d] = {}
        split_features[init_type][d] = {}
        #split_values[init_type][d] = {}
        
        max_depth_dict[init_type][d] = {}
        
        for GA_strat in GA_strats:
            
            filename = init_source + f"list{d}_depth_{GA_strat}_gen.json"
            
            with open(filename) as json_file:
                data = json.load(json_file) # contains a sublist for each generation; 
                                            # sublists each contain 1320 tuples, where 1st element is the model, and 2nd the reward it produced
            
            last_gen_data = data[-1]
            best_5_models[init_type][d][str(GA_strat)] = sorted(last_gen_data, key=lambda x: x[1], reverse=True)[:5]
            
            # sorted(avg_total_rewards.items(), key=lambda x: x[1], reverse=True)
            best_5_model_objects[init_type][d][str(GA_strat)] = []
            split_features[init_type][d][str(GA_strat)] = []
            #split_values[init_type][d][str(GA_strat)] = []
            max_depth_dict[init_type][d][str(GA_strat)] = []
            
            
            # for model_name, best_rew in best_5_models[init_type][d][str(GA_strat)]:
            #     #print(best_rew)
            #     path_to_model = model_sources[init_type] + f"list{d}_depth_{GA_strat}_GA_{which_generation}_gen_{model_name}"
                
            #     with open(path_to_model, 'rb') as file:
            #         loaded_model = pickle.load(file)
                    
                
            #     ###################################
            #     #### run final 50-episode test ####
            #     ###################################
            #     N_episodes = 10

            #     total_reward_dummy = np.array([])
                
            #     for i in range(N_episodes):
            #         obs = env.reset()
            #         done = False
                    
            #         while not done:
            #             std_obs = obs
            #             action = loaded_model.predict_action(std_obs)

            #             obs, reward, done, _ = env.step(action)
            #             total_reward_dummy = np.append(total_reward_dummy, [reward])
                                            
            #     # now we have the best 5 models and their final performance for each init type, depth and GA strat
            #     best_5_model_objects[init_type][d][str(GA_strat)].append((loaded_model, np.average(total_reward_dummy)))
                #split_features[init_type][d][str(GA_strat)].append([get_all_split_features(loaded_model)])
                #split_values[init_type][d][str(GA_strat)].append([get_all_split_values(loaded_model)])
                #max_depth_dict[init_type][d][str(GA_strat)].append(get_max_depth(loaded_model))
                         

In [6]:
import pandas as pd

tables = {}
for init_type, init_type_data in best_5_models.items():
    table_data = {}
    for d, d_data in init_type_data.items():
        for GA_strat, tuples in d_data.items():
            max_float = max(t[1] for t in tuples)
            table_data.setdefault(d, {})[GA_strat] = max_float

    # Convert the dictionary to a DataFrame
    tables[init_type] = pd.DataFrame(table_data)

# Print or use the tables as needed
for init_type, table in tables.items():
    print(f"Table for {init_type}:\n{table}\n")

Table for random:
                    3         5         10        13
[50, 30, 20]  0.840899  0.852610  0.840899  0.852610
[40, 40, 20]  0.840227  0.881939  0.840227  0.881939
[30, 40, 30]  0.844699  0.833790  0.844699  0.833790
[20, 50, 30]  0.843766  0.846767  0.843766  0.890671
[10, 50, 40]  0.840979  0.885388  0.840979  0.885388

Table for half_half:
                    3         5         10        13
[50, 30, 20]  0.848432  0.858084  0.884147  0.825404
[40, 40, 20]  0.840227  0.881939  0.852574  0.838547
[30, 40, 30]  0.844699  0.860554  0.889757  0.862833
[20, 50, 30]  0.843766  0.942868  0.841350  0.836876
[10, 50, 40]  0.840979  0.846089  0.840176  0.837421

Table for full_heuristic:
                    3         5         10        13
[50, 30, 20]  0.807047  0.834280  0.868323  0.880887
[40, 40, 20]  0.827749  0.867596  0.912032  0.889922
[30, 40, 30]  0.854648  0.887658  0.837331  0.888835
[20, 50, 30]  0.866348  0.903953  0.878274  0.909002
[10, 50, 40]  0.909659  0.920159

In [7]:
tables['full_heuristic']

,3,5,10,13
"[50, 30, 20]",0.807047,0.834280,0.868323,0.880887
"[40, 40, 20]",0.827749,0.867596,0.912032,0.889922
"[30, 40, 30]",0.854648,0.887658,0.837331,0.888835
"[20, 50, 30]",0.866348,0.903953,0.878274,0.909002
"[10, 50, 40]",0.909659,0.920159,0.923882,0.840091
